In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf

  Activating environment at `C:\Users\hilld\Documents\git\ocrl\ocrl_quadruped_mpc\nonlinear_approach\Project.toml`


## Julia note:

incorrect:


`x_l[idx.x[i]][2] = 0 # this does not change x_l`


correct:


`x_l[idx.x[i][2]] = 0 # this changes x_l`


It should always be `v[index] = new_val` if I want to update v with `new_val` at `index`.

In [2]:
let 
    
    # vector we want to modify 
    Z = randn(5)
    
    # original value of Z so we can check if we are changing it
    Z_original = 1 * Z 
    
    # index range we are considering
    idx_x = 1:3 
    
    # this does NOT change Z 
    Z[idx_x][2] = 0 
    
    # we can prove this 
    @show norm(Z - Z_original) 
    
    # this DOES change Z 
    Z[idx_x[2]] = 0 
    
    # we can prove this 
    @show norm(Z - Z_original) 
    
    
end

norm(Z - Z_original) = 0.0
norm(Z - Z_original) = 0.9184675568126108


0.9184675568126108

In [3]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","walker.jl"))

LoadError: SystemError: opening file "C:\\Users\\hilld\\Documents\\git\\ocrl\\ocrl_quadruped_mpc\\nonlinear_approach\\utils\\fmincon.jl": No such file or directory

![](walker.gif)
(If nothing loads here, check out `walker.gif` in the repo)

**NOTE: This question will have long outputs for each cell, remember you can use `cell -> all output -> toggle scrolling` to better see it all**

# Q2: Hybrid Trajectory Optimization  (60 pts)
In this problem you'll use a direct method to optimize a walking trajectory for a simple biped model, using the hybrid dynamics formulation. You'll pre-specify a gait sequence and solve the problem using Ipopt. Your final solution should look like the video above.

## The Dynamics
Our system is modeled as 5 point masses: one for the body and one for each foot. The state is defined as the x and y positions and velocities of these masses, for a total of 6 degrees of freedom and 20 states. We will label the position and velocity of each body with the following notation:
$$ \begin{align} 
r^{(b)} &= \begin{bmatrix} p_x^{(b)} \\ p_y^{(b)} \end{bmatrix} & v^{(b)} &= \begin{bmatrix} v_x^{(b)} \\ v_y^{(b)} \end{bmatrix}\\
r^{(1)} &= \begin{bmatrix} p_x^{(1)} \\ p_y^{(1)} \end{bmatrix} & v^{(1)} &= \begin{bmatrix} v_x^{(1)}\\
v_y^{(1)} \end{bmatrix}\\
r^{(2)} &= \begin{bmatrix} p_x^{(2)} \\ p_y^{(2)} \end{bmatrix} & v^{(2)} &= \begin{bmatrix} v_x^{(2)}\\
v_y^{(2)} \end{bmatrix}\\
r^{(3)} &= \begin{bmatrix} p_x^{(3)} \\ p_y^{(3)} \end{bmatrix} & v^{(3)} &= \begin{bmatrix} v_x^{(3)}\\
v_y^{(3)} \end{bmatrix}\\
r^{(4)} &= \begin{bmatrix} p_x^{(4)} \\ p_y^{(4)} \end{bmatrix} & v^{(4)} &= \begin{bmatrix} v_x^{(4)}\\
v_y^{(4)} \end{bmatrix}

\end{align}$$
Each leg is connected to the body with prismatic joints. The system has three control inputs: a force along each leg, and the torque between each pair of legs.

The state and control vectors are ordered as follows:

$$ x = \begin{bmatrix} 
    p_x^{(b)} \\ p_y^{(b)} \\ p_x^{(1)} \\ p_y^{(1)} \\ p_x^{(2)} \\ p_y^{(2)} \\ p_x^{(3)} \\ p_y^{(3)} \\ p_x^{(4)} \\ p_y^{(4)} \\
    v_x^{(b)} \\ v_y^{(b)} \\ v_x^{(1)} \\ v_y^{(1)} \\ v_x^{(2)} \\ v_y^{(2)} \\ v_x^{(3)} \\ v_y^{(3)} \\ v_x^{(4)} \\ v_y^{(4)} \\
\end{bmatrix} \quad
u = \begin{bmatrix} F^{(1)} \\ F^{(2)} \\ F^{(3)} \\ F^{(4)} \\ \tau \end{bmatrix}
or
u = \begin{bmatrix} F^{(1)} \\ F^{(2)} \\ \tau^{(1)} \\ F^{(3)} \\ F^{(4)} \\ \tau^{(2)} \end{bmatrix}
$$
where e.g. $p_x^{(b)}$ is the $x$ position of the body, $v_y^{(i)}$ is the $y$ velocity of foot $i$, $F^{(i)}$ is the force along leg $i$, and $\tau$ is the torque between the legs. 

The continuous time dynamics and jump maps for the two stances are shown below:

In [4]:
function stance134_dynamics(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 1,3, 4 is in contact with the ground 
    
    mb,mf = model.mb, model.mf
    g = model.g

    M = Diagonal([mb mb mf mf mf mf mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    rf3 = x[7:8]   # position of foot 3
    rf4 = x[9:10]   # position of foot 4
    v   = x[11:20]  # velocities
    
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    ℓ3x = (rb[1]-rf3[1])/norm(rb-rf3)
    ℓ3y = (rb[2]-rf3[2])/norm(rb-rf3)
    ℓ4x = (rb[1]-rf4[1])/norm(rb-rf4)
    ℓ4y = (rb[2]-rf4[2])/norm(rb-rf4)
    
    # try 1 one foot at a time 
    B = [ℓ1x  ℓ2x  ℓ3x  ℓ4x  ℓ1y+ℓ4y-ℓ2y-ℓ3y;
         ℓ1y  ℓ2y  ℓ3y  ℓ4y  ℓ2x+ℓ3x-ℓ1x-ℓ4y;
          0    0    0    0     0;
          0    0    0    0     0;
          0  -ℓ2x   0    0    ℓ2y;
          0  -ℓ2y   0    0   -ℓ2x;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0]
    
    v̇ = [0; -g; 0; 0; 0; -g; 0; 0; 0; -g] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end


function stance123_dynamics(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 1,2,3 is in contact with the ground 
    
    mb,mf = model.mb, model.mf
    g = model.g

    M = Diagonal([mb mb mf mf mf mf mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    rf3 = x[7:8]   # position of foot 3
    rf4 = x[9:10]   # position of foot 4
    v   = x[11:20]  # velocities
    
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    ℓ3x = (rb[1]-rf3[1])/norm(rb-rf3)
    ℓ3y = (rb[2]-rf3[2])/norm(rb-rf3)
    ℓ4x = (rb[1]-rf4[1])/norm(rb-rf4)
    ℓ4y = (rb[2]-rf4[2])/norm(rb-rf4)
    
  
    B = [ℓ1x  ℓ2x  ℓ3x  ℓ4x  ℓ1y+ℓ4y-ℓ2y-ℓ3y;
         ℓ1y  ℓ2y  ℓ3y  ℓ4y  ℓ2x+ℓ3x-ℓ1x-ℓ4y;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0  -ℓ4x   -ℓ4y;
          0    0    0  -ℓ4y    ℓ4x]
    
    v̇ = [0; -g; 0; 0; 0; -g; 0; 0; 0; -g] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

function stance234_dynamics(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 2,3,4 is in contact with the ground 
    
    mb,mf = model.mb, model.mf
    g = model.g

    M = Diagonal([mb mb mf mf mf mf mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    rf3 = x[7:8]   # position of foot 3
    rf4 = x[9:10]   # position of foot 4
    v   = x[11:20]  # velocities
    
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    ℓ3x = (rb[1]-rf3[1])/norm(rb-rf3)
    ℓ3y = (rb[2]-rf3[2])/norm(rb-rf3)
    ℓ4x = (rb[1]-rf4[1])/norm(rb-rf4)
    ℓ4y = (rb[2]-rf4[2])/norm(rb-rf4)
    
  
    B = [ℓ1x  ℓ2x  ℓ3x  ℓ4x  ℓ1y+ℓ4y-ℓ2y-ℓ3y;
         ℓ1y  ℓ2y  ℓ3y  ℓ4y  ℓ2x+ℓ3x-ℓ1x-ℓ4y;
        -ℓ1x   0    0    0    -ℓ1y;
        -ℓ1y   0    0    0     ℓ1x;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0]
    
    v̇ = [0; -g; 0; 0; 0; -g; 0; 0; 0; -g] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

function stance124_dynamics(model::NamedTuple, x::Vector, u::Vector)
    # dynamics when foot 2,1,4 is in contact with the ground 
    
    mb,mf = model.mb, model.mf
    g = model.g

    M = Diagonal([mb mb mf mf mf mf mf mf mf mf])
    
    rb  = x[1:2]   # position of the body
    rf1 = x[3:4]   # position of foot 1
    rf2 = x[5:6]   # position of foot 2
    rf3 = x[7:8]   # position of foot 3
    rf4 = x[9:10]   # position of foot 4
    v   = x[11:20]  # velocities
    
    
    ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
    ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
    ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
    ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    ℓ3x = (rb[1]-rf3[1])/norm(rb-rf3)
    ℓ3y = (rb[2]-rf3[2])/norm(rb-rf3)
    ℓ4x = (rb[1]-rf4[1])/norm(rb-rf4)
    ℓ4y = (rb[2]-rf4[2])/norm(rb-rf4)
    
  
    B = [ℓ1x  ℓ2x  ℓ3x  ℓ4x  ℓ1y+ℓ4y-ℓ2y-ℓ3y;
         ℓ1y  ℓ2y  ℓ3y  ℓ4y  ℓ2x+ℓ3x-ℓ1x-ℓ4y;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0    0    0     0;
          0    0  -ℓ3x   0     ℓ3y;
          0    0  -ℓ3y   0    -ℓ3x;
          0    0    0    0     0;
          0    0    0    0     0]
    
    v̇ = [0; -g; 0; 0; 0; -g; 0; 0; 0; -g] + M\(B*u)
    
    ẋ = [v; v̇]
    
    return ẋ
end

# function stance2_dynamics(model::NamedTuple, x::Vector, u::Vector)
#     # dynamics when foot 2 is in contact with the ground 
    
#     mb,mf = model.mb, model.mf
#     g = model.g
#     M = Diagonal([mb mb mf mf mf mf])
    
#     rb  = x[1:2]   # position of the body
#     rf1 = x[3:4]   # position of foot 1
#     rf2 = x[5:6]   # position of foot 2
#     v   = x[7:12]  # velocities
    
#     ℓ1x = (rb[1]-rf1[1])/norm(rb-rf1)
#     ℓ1y = (rb[2]-rf1[2])/norm(rb-rf1)
#     ℓ2x = (rb[1]-rf2[1])/norm(rb-rf2)
#     ℓ2y = (rb[2]-rf2[2])/norm(rb-rf2)
    
#     B = [ℓ1x  ℓ2x  ℓ1y-ℓ2y;
#          ℓ1y  ℓ2y  ℓ2x-ℓ1x;
#         -ℓ1x   0  -ℓ1y;
#         -ℓ1y   0   ℓ1x;
#           0    0    0;
#           0    0    0]
    
#     v̇ = [0; -g; 0; -g; 0; 0] + M\(B*u)
    
#     ẋ = [v; v̇]
    
#     return ẋ
# end

function jump1_map(x)
    # foot 1,3,4 experiences inelastic collision
    xn = [x[1:12]; 0.0; 0.0; x[15:16]; 0.0; 0.0; 0.0; 0.0]
    return xn
end

function jump2_map(x)
    # foot 1,2,3 experiences inelastic collision
    xn = [x[1:12]; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; x[19:20]]
    return xn
end

function jump3_map(x)
    # foot 2,3,4 experiences inelastic collision
    xn = [x[1:14]; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]
    return xn
end

function jump4_map(x)
    # foot 1,2,4 experiences inelastic collision
    xn = [x[1:12]; 0.0; 0.0; 0.0; 0.0; x[17:18]; 0.0; 0.0]
    return xn
end
function rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end  

rk4 (generic function with 1 method)

We are setting up this problem by scheduling out the contact sequence. To do this, we will define the following sets:

$$ \begin{align} 
\mathcal{M}_134 &= \{1\text{:}5,21\text{:}25,41\text{:}45,61\text{:}65\} \\
\mathcal{M}_123 &= \{6\text{:}10,26\text{:}30,46\text{:}50,66\text{:}70\} \\
\mathcal{M}_234 &= \{11\text{:}15,31\text{:}35,51\text{:}55,71\text{:}75\} \\
\mathcal{M}_124 &= \{16\text{:}20,36\text{:}40,56\text{:}60,76\text{:}80\}
\end{align}$$

where $\mathcal{M}_1$ contains the time steps when foot 1,3,4 is pinned to the ground (`stance134_dynamics`), and $\mathcal{M}_2$ contains the time steps when foot 1,2,3 is pinned to the ground (`stance123_dynamics`). $\mathcal{M}_3$ contains the time steps when foot 2,3,4 is pinned to the ground (`stance234_dynamics`), and $\mathcal{M}_4$ contains the time steps when foot 1,2,4 is pinned to the ground (`stance124_dynamics`). The jump map sets $\mathcal{J}_1$ and $\mathcal{J}_2$ and $\mathcal{J}_3$ and $\mathcal{J}_4$ are the indices where the mode of the next time step is different than the current, i.e. $\mathcal{J}_i \equiv \{k+1 \notin \mathcal{M}_i \; | \; k \in \mathcal{M}_i\}$. We can write these out explicitly as the following:

$$ \begin{align} 
\mathcal{J}_1 &= \{5,25,45,65\} \\
\mathcal{J}_2 &= \{10,30,50,70\} \\
\mathcal{J}_3 &= \{15,35,55,75\} \\
\mathcal{J}_4 &= \{20,40,60,80\}
\end{align}$$

Another term you will see is set subtraction, or $\mathcal{M}_i \setminus \mathcal{J}_i$. This just means that if  $k \in \mathcal{M}_i \setminus \mathcal{J}_i$, then $k$ is in $\mathcal{M}_i$ but not in $\mathcal{J}_i$. 

We will make use of the following Julia code for determining which set an index belongs to:

In [5]:
let 
    M134 = vcat([ (i-1)*20      .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    M123 = vcat([((i-1)*20 + 5) .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    M234 = vcat([ (2i-1)*10      .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    M124 = vcat([((2i-1)*10 + 5) .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    J1 = [5,25,45,65]
    J2 = [10,30,50,70]
    J3 = [15,35,55,75]
    J4 = [20,40,60,80]

    # Q = diagm([1; 10; fill(1.0, 8); 1; 10; fill(1.0, 8)]);
    # R = diagm(fill(1e-3,6))
    # @show R
    # @show Q
    tf = 7.9 
    dt = 0.1 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    @show N
    @show M134
    @show M123 
    @show M234
    @show M124 
    @show J1
    @show J2 
    # @show  (5 in M1) # show if 5 is in M1 
    # @show  (5 in J1) # show if 5 is in J1 
    # @show !(5 in M1) # show is 5 is not in M1 
    
    # @show (5 in M1) && !(5 in J1) # 5 in M1 but not J1 (5 ∈ M_1 \ J1)
    
end

N = 80
M134 = [1, 2, 3, 4, 5, 21, 22, 23, 24, 25, 41, 42, 43, 44, 45, 61, 62, 63, 64, 65]
M123 = [6, 7, 8, 9, 10, 26, 27, 28, 29, 30, 46, 47, 48, 49, 50, 66, 67, 68, 69, 70]
M234 = [11, 12, 13, 14, 15, 31, 32, 33, 34, 35, 51, 52, 53, 54, 55, 71, 72, 73, 74, 75]
M124 = [16, 17, 18, 19, 20, 36, 37, 38, 39, 40, 56, 57, 58, 59, 60, 76, 77, 78, 79, 80]
J1 = [5, 25, 45, 65]
J2 = [10, 30, 50, 70]


4-element Vector{Int64}:
 10
 30
 50
 70

We are now going to setup and solve a constrained nonlinear program.  The optimization problem looks complicated but each piece should make sense and be relatively straightforward to implement. First we have the following LQR cost function that will track $x_{ref}$ (`Xref`) and $u_{ref}$ (`Uref`):

$$  J(x_{1:N},u_{1:N-1}) = \sum_{i=1}^{N-1} \bigg[ \frac{1}{2} (x_i - x_{ref,i})^TQ(x_i - x_{ref,i}) + \frac{1}{2} (u_i-u_{ref,i})^TR(u_i-u_{ref,i}) \bigg] + \frac{1}{2}(x_N - x_{ref,N})^TQ_f(x_N - x_{ref,N})$$

Which goes into the following full optimization problem:
   $$ \begin{align} \min_{x_{1:N},u_{1:N-1}} \quad & J(x_{1:N},u_{1:N-1}) & \\ 
 \text{st} \quad &  x_1 = x_{ic} & \tag{1}\\
 \quad & x_N = x_{g} &\tag{2}\\
 &  x_{k+1} = f_1(x_k,u_k)  &\text{for } k \in  \mathcal{M}_134 \setminus \mathcal{J}_1 \tag{3}\\ 
 &  x_{k+1} = f_2(x_k,u_k)  &\text{for } k \in  \mathcal{M}_123 \setminus \mathcal{J}_2 \tag{4}\\ 
 &  x_{k+1} = f_3(x_k,u_k)  &\text{for } k \in  \mathcal{M}_234 \setminus \mathcal{J}_3 \tag{5}\\ 
 &  x_{k+1} = f_4(x_k,u_k)  &\text{for } k \in  \mathcal{M}_124 \setminus \mathcal{J}_4 \tag{6}\\ 
 &  x_{k+1} = g_2(f_1(x_k,u_k))  &\text{for } k \in   \mathcal{J}_1 \tag{7}\\ 
 &  x_{k+1} = g_3(f_2(x_k,u_k))  &\text{for } k \in   \mathcal{J}_2 \tag{8}\\
 &  x_{k+1} = g_4(f_3(x_k,u_k))  &\text{for } k \in   \mathcal{J}_3 \tag{9}\\ 
 &  x_{k+1} = g_1(f_4(x_k,u_k))  &\text{for } k \in   \mathcal{J}_4 \tag{10}\\ 
 &  x_{k}[4,8,10] = 0  &\text{for } k \in   \mathcal{M}_134 \tag{11}\\ 
 &  x_{k}[4,6,8] = 0  &\text{for } k \in   \mathcal{M}_123  \tag{12}\\
  &  x_{k}[6,8,10] = 0  &\text{for } k \in   \mathcal{M}_234 \tag{13}\\ 
 &  x_{k}[4,6,10] = 0  &\text{for } k \in   \mathcal{M}_124  \tag{14}\\ 
 &  0.5 \leq \| r^{(b)}_k - r^{(1)}_k\|_2 \leq 1.5 & \text{for } k \in [1, N]\tag{15} \\
  &  0.5 \leq \| r^{(b)}_k - r^{(2)}_k\|_2 \leq 1.5 & \text{for } k \in [1, N]\tag{16} \\
  &  0.5 \leq \| r^{(b)}_k - r^{(3)}_k\|_2 \leq 1.5 & \text{for } k \in [1, N]\tag{17} \\
  &  0.5 \leq \| r^{(b)}_k - r^{(4)}_k\|_2 \leq 1.5 & \text{for } k \in [1, N]\tag{18} \\
 &  x_{k}[2,4,6,8,10] \geq 0 & \text{for } k \in [1, N]\tag{19}
 \end{align}$$
 


And here we have the list of mathematical functions to the Julia function names:
- $f_1$ is `stance134_dynamics` + `rk4`
- $f_2$ is `stance123_dynamics` + `rk4` and so on
- $g_1$ is `jump1_map`
- $g_2$ is `jump2_map`

For instance, $g_2(f_1(x_k,u_k))$ is `jump2_map(rk4(model, stance1_dynamics, xk, uk, dt))`

Remember that $r^{(b)}$ is defined above.

In [6]:
function reference_trajectory(model, xic, xg, dt, N)
    # creates a reference Xref and Uref for walker 
    
    Uref = [[model.mb*model.g*0.25;model.mb*model.g*0.25;model.mb*model.g*0.25;model.mb*model.g*0.25;0] for i = 1:(N-1)]
    
    Xref = [zeros(20) for i = 1:N]
    
    horiz_v = (3/N)/dt 
    xs = range(-1.5, 1.5, length = N)
    Xref[1] = 1*xic 
    Xref[N] = 1*xg 
    
    for i = 2:(N-1) 
        Xref[i] = [xs[i],1,xs[i],0,xs[i],0,xs[i],0,xs[i],0,horiz_v,0,horiz_v,0,horiz_v,0,horiz_v,0,horiz_v,0]
    end
        
    return Xref, Uref
end

reference_trajectory (generic function with 1 method)

To solve this problem with Ipopt and `fmincon`, we are going to concatenate all of our $x$'s and $u$'s into one vector (same as HW3Q1):

$$ Z = \begin{bmatrix}x_1 \\ u_1 \\ x_2 \\ u_2 \\ \vdots \\ x_{N-1} \\ u_{N-1} \\ x_N \end{bmatrix} \in \mathbb{R}^{N \cdot nx + (N-1)\cdot nu} $$ 

where $x \in \mathbb{R}^{nx}$ and $u \in \mathbb{R}^{nu}$. Below we will provide useful indexing guide in `create_idx` to help you deal with $Z$.  Remember that the API for `fmincon` (that we used in HW3Q1) is the following:
$$ \begin{align} \min_{z} \quad & \ell(z) & \text{cost function}\\ 
 \text{st} \quad & c_{eq}(z) = 0 & \text{equality constraint}\\
 & c_L \leq c_{ineq}(z) \leq c_U & \text{inequality constraint}\\
 & z_L \leq z \leq z_U & \text{primal bound constraint}
 \end{align}$$
 
 Template code has been given to solve this problem but you should feel free to do whatever is easiest for you, as long as you get the trajectory shown in the animation `walker.gif` and pass tests. 

In [7]:
# feel free to solve this problem however you like, below is a template for a 
# good way to start. 

function create_idx(nx,nu,N)
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # and stacked dynamics constraints of size nx are 
    # c[idx.c[i]] = <dynamics constraint at time step i>
    #
    # feel free to use/not use this 
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function walker_cost(params::NamedTuple, Z::Vector)::Real
    # cost function 
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref,Uref = params.Xref, params.Uref
    
    # TODO: input walker LQR cost 
    
    J = 0 
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 0.5*(xi - Xref[i])'*Q*(xi - Xref[i]) + 0.5*(ui - Uref[i])'*R*(ui - Uref[i])
        # J += 0.5*(xi - xg)'*Q*(xi - xg) + 0.5*(ui )'*R*(ui)

    end
    
    # dont forget terminal cost 
    J += 0.5*(Z[idx.x[N]]-xg)'*Qf*(Z[idx.x[N]]-xg)

        
    return J 
end

function walker_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt 
    M134, M123, M234, M124 = params.M134, params.M123, params.M234, params.M124  
    J1, J2, J3, J4 = params.J1, params.J2, params.J3, params.J4 
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    # TODO: input walker dynamics constraints (constraints 3-6 in the opti problem)
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # (5 in M1) && !(5 in J1) # 5 in M1 but not J1 (5 ∈ M_1 \ J1)
        if (i in M134) # && !(i in J1)
            c[idx.c[i]] = xip1 - rk4(model, stance134_dynamics, xi, ui, dt)
        end
        if (i in M123) # && !(i in J2)
            c[idx.c[i]] = xip1 - rk4(model, stance123_dynamics, xi, ui, dt)
        end
        if (i in M234) # && !(i in J1)
            c[idx.c[i]] = xip1 - rk4(model, stance234_dynamics, xi, ui, dt)
        end
        if (i in M124) # && !(i in J2)
            c[idx.c[i]] = xip1 - rk4(model, stance124_dynamics, xi, ui, dt)
        end
        if (i in J1)
            c[idx.c[i]] = xip1 - jump2_map(rk4(model, stance134_dynamics, xi, ui, dt))
        end
        if (i in J2)
            c[idx.c[i]] = xip1 - jump3_map(rk4(model, stance123_dynamics, xi, ui, dt))
        end
        if (i in J3)
            c[idx.c[i]] = xip1 - jump4_map(rk4(model, stance234_dynamics, xi, ui, dt))
        end
        if (i in J4)
            c[idx.c[i]] = xip1 - jump1_map(rk4(model, stance124_dynamics, xi, ui, dt))
        end
    end

    return c 
end

function walker_stance_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    M134, M123, M234, M124 = params.M134, params.M123, params.M234, params.M124  
    J1, J2, J3, J4 = params.J1, params.J2, params.J3, params.J4 
    
    model = params.model 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), N)
    
    # TODO: add walker stance constraints (constraints 7-8 in the opti problem)
    for i = 1:(N)
        xi = Z[idx.x[i]]
        # ui = Z[idx.u[i]] 
        # xip1 = Z[idx.x[i+1]]
        
        if (i in M134)
            c[i] = xi[4]
        end
        if (i in M123)
            c[i] = xi[6]
        end
        if (i in M234)
            c[i] = xi[8]
        end
        if (i in M124)
            c[i] = xi[10]
        end
    end

    return c
end
    
function walker_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    # TODO: stack up all of our equality constraints 
    
    # should be length 2*nx + (N-1)*nx + N 
    # inital condition constraint (nx)       (constraint 1)
    # terminal constraint         (nx)       (constraint 2)
    # dynamics constraints        (N-1)*nx   (constraint 3-6)
    # stance constraint           N          (constraint 7-8)
    initial_cond = Z[idx.x[1]] - xic
    terminal_cond = Z[idx.x[N]] - xg
    dynamics_cond = walker_dynamics_constraints(params, Z)
    stance_cond = walker_stance_constraint(params, Z)

    return [initial_cond; terminal_cond; dynamics_cond; stance_cond]
end

function walker_inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    # M1, M2 = params.M1, params.M2 
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 4*N)
    
    # TODO: add the length constraints shown in constraints (9-10)
    # there are 2*N constraints here 
    for i = 1:(N)
        xi = Z[idx.x[i]]
        # ui = Z[idx.u[i]] 
        # xip1 = Z[idx.x[i+1]]
        rb  = xi[1:2]   # position of the body
        rf1 = xi[3:4]   # position of foot 1
        rf2 = xi[5:6]   # position of foot 2
        rf3 = xi[7:8]   # position of foot 3
        rf4 = xi[9:10]   # position of foot 4
    
        
        # i = 1,5,9,13,..
        id_1 = 4*i-3
        # i = 2,6,10,14,..
        id_2 = 4*i-2
        # i = 3,7,11,15,..
        id_3 = 4*i-1
        # i = 4,8,12,16,..
        id_4 = 4*i
        # c[id_1] = norm(rb - rf1)^2
        # c[id_2] = norm(rb - rf2)^2
        c[id_1] = norm(rb - rf1)
        c[id_2] = norm(rb - rf2)
        c[id_3] = norm(rb - rf3)
        c[id_4] = norm(rb - rf4)
    end

    return c
end

walker_inequality_constraint (generic function with 1 method)

In [8]:
@testset "walker trajectory optimization" begin 
    
    # dynamics parameters 
    model = (g = 9.81, mb= 5.0, mf = 1.0, ℓ_min = 0.5, ℓ_max = 1.5)
    
    # problem size 
    nx = 20 #12 
    nu = 5 #3 
    tf = 7.9 #4.4 
    dt = 0.1 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # initial and goal states 
    xic = [-1.5;1;-1.0;0;-1.0;0;-2.0;0;-2.0;0;0;0;0;0;0;0;0;0;0;0]
    xg =  [1.5;1;2.0;0;2.0;0;1.0;0;1.0;0;0;0;0;0;0;0;0;0;0;0]
    
    # (###############################CCCCCCCCCCHHHHHHHHHHEEEEEEEEECCCCCCCCCCCCKKKKKKKKKKKKKKKKKK#################################)
    # index sets 
    # M1 = vcat([ (i-1)*10      .+ (1:5)   for i = 1:5]...)
    # M2 = vcat([((i-1)*10 + 5) .+ (1:5)   for i = 1:4]...)
    # J1 = [5,15,25,35]
    # J2 = [10,20,30,40]
    M134 = vcat([ (i-1)*20      .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    M123 = vcat([((i-1)*20 + 5) .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    M234 = vcat([ (2i-1)*10      .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    M124 = vcat([((2i-1)*10 + 5) .+ (1:5)   for i = 1:4]...) # stack the set into a vector
    J1 = [5,25,45,65]
    J2 = [10,30,50,70]
    J3 = [15,35,55,75]
    J4 = [20,40,60,80]
    # (###############################CCCCCCCCCCHHHHHHHHHHEEEEEEEEECCCCCCCCCCCCKKKKKKKKKKKKKKKKKK#################################)
    
    # reference trajectory 
    Xref, Uref = reference_trajectory(model, xic, xg, dt, N)
    
    # LQR cost function (tracking Xref, Uref)
    Q = diagm([1; 10; fill(1.0, 8); 1; 10; fill(1.0, 8)]);
    R = diagm(fill(1e-3,5))
    Qf = 1*Q;
    
    # create indexing utilities 
    idx = create_idx(nx,nu,N)
    
    # put everything useful in params 
    params = (
        model = model, 
        nx = nx,
        nu = nu,
        tf = tf, 
        dt = dt, 
        t_vec = t_vec,
        N = N,
        M134 = M134,
        M123 = M123,
        M234 = M234,
        M124 = M124,
        J1 = J1, 
        J2 = J2,
        J3 = J3, 
        J4 = J4,
        xic = xic, 
        xg = xg,
        idx = idx,
        Q = Q, R = R, Qf = Qf,
        Xref = Xref, 
        Uref = Uref
    )
    # (###############################CCCCCCCCCCHHHHHHHHHHEEEEEEEEECCCCCCCCCCCCKKKKKKKKKKKKKKKKKK#################################)
    # TODO: primal bounds (constraint 11)
    x_l = -Inf*ones(idx.nz) # update this 
    x_u =  Inf*ones(idx.nz) # update this 
    for i = 1:(N)
        x_l[idx.x[i][2]] = 0
        x_l[idx.x[i][4]] = 0
        x_l[idx.x[i][6]] = 0
        x_l[idx.x[i][8]] = 0
        x_l[idx.x[i][10]] = 0
    end

    
    # TODO: inequality constraint bounds
    # c_l = 0.25*ones(2*N) #0.5*ones(2*N) # update this 
    # c_u = 2.25*ones(2*N) #1.5*ones(2*N) # update this 
    c_l = 0.5*ones(4*N) # update this 
    c_u = 1.5*ones(4*N) # update this 
    # (###############################CCCCCCCCCCHHHHHHHHHHEEEEEEEEECCCCCCCCCCCCKKKKKKKKKKKKKKKKKK#################################)

    # TODO: initialize z0 with the reference Xref, Uref 
    z0 = zeros(idx.nz) # update this 
    print("debuggingggg 1")
    # print(Uref)
    for i = 1:(N-1)
        z0[idx.x[i]] = Xref[i]
        z0[idx.u[i]] = Uref[i]
    end
    z0[idx.x[N]] = Xref[N]
    print("debuggingggg 2")
    # adding a little noise to the initial guess is a good idea 
    z0 = z0 + (1e-6)*randn(idx.nz)
    
    # diff_type = :finite #:auto
    diff_type = :auto 
    
    Z = fmincon(walker_cost,walker_equality_constraint,walker_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-3, c_tol = 1e-3, max_iters = 10_000, verbose = true)

    print("debuggingggg 3")
    #######################################################################
    # cd = walker_dynamics_constraints(params, Z)
    # cs = walker_stance_constraint(params, Z)
    # ce = walker_equality_constraint(params, Z)
    # cie = walker_inequality_constraint(params, Z)

    # @show cd
    # @show cs
    # @show ce
    # @show cie
    #######################################################################
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    # ------------plotting--------------
    Xm = hcat(X...)
    Um = hcat(U...)
    
    plot(Xm[1,:],Xm[2,:], label = "body")
    plot!(Xm[3,:],Xm[4,:], label = "leg 1")
    plot!(Xm[5,:],Xm[6,:], label = "leg 2")
    plot!(Xm[7,:],Xm[8,:], label = "leg 3")
    display(plot!(Xm[9,:],Xm[10,:], label = "leg 4",xlabel = "x (m)",
                  ylabel = "y (m)", title = "Body Positions"))

    display(plot(t_vec[1:end-1], Um',xlabel = "time (s)", ylabel = "U",
                 label = ["F1" "F2" "F3" "F4" "τ2"], title = "Controls"))
    
    # ---------animation-------------
    vis = Visualizer()
    build_walker!(vis, model::NamedTuple)
    anim = mc.Animation(floor(Int,1/dt))
    for k = 1:N
        mc.atframe(anim, k) do
            update_walker_pose!(vis, model::NamedTuple, X[k])
        end
    end
    mc.setanimation!(vis, anim)
    display(render(vis))
    
    # --------testing---------------- 
    
    # initial and terminal states 
    @test norm(X[1] - xic,Inf) <= 1e-3 
    @test norm(X[end] - xg,Inf) <= 1e-3
    
    for x in X  
        
        # distance between bodies
        rb = x[1:2]
        rf1 = x[3:4]
        rf2 = x[5:6]
        @test (0.5 - 1e-3) <= norm(rb-rf1) <= (1.5 + 1e-3)
        @test (0.5 - 1e-3) <= norm(rb-rf2) <= (1.5 + 1e-3)
        
        # no two feet moving at once 
        v1 = x[9:10]
        v2 = x[11:12] 
        @test min(norm(v1,Inf),norm(v2,Inf)) <= 1e-3 
        
        # check everything above the surface 
        @test x[2] >= (0 - 1e-3)
        @test x[4] >= (0 - 1e-3)
        @test x[6] >= (0 - 1e-3)

        #adding test cases
        # @show x[4]
        # @show x[6]
        
    end
    
end

debuggingggg 1debuggingggg 2walker trajectory optimization: Error During Test at In[8]:1
  Got exception outside of a @test
  UndefVarError: fmincon not defined
  Stacktrace:
    [1] macro expansion
      @ In[8]:105 [inlined]
    [2] macro expansion
      @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\Test\src\Test.jl:1151 [inlined]
    [3] top-level scope
      @ In[8]:4
    [4] eval
      @ .\boot.jl:360 [inlined]
    [5] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
      @ Base .\loading.jl:1116
    [6] softscope_include_string(m::Module, code::String, filename::String)
      @ SoftGlobalScope C:\Users\hilld\.julia\packages\SoftGlobalScope\u4UzH\src\SoftGlobalScope.jl:65
    [7] execute_request(socket::ZMQ.Socket, msg::IJulia.Msg)
      @ IJulia C:\Users\hilld\.julia\packages\IJulia\Vo51o\src\execute_request.jl:67
    [8] #invokelatest#2
      @ .\essentials.jl:708 [inlined]
    [9] invokelatest
      @ .\essenti

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.[39m